We use Custom tokenization for the process as we convert the answers labels as well in fly by while tokenzing the questions and answers both

In [23]:
from transformers import pipeline
qa_pipeline = pipeline("question-answering", model="distilbert-base-uncased-distilled-squad")
context = """
Transformers are a type of neural network architecture that has proven very effective
in natural language processing tasks such as translation, summarization, and question answering.
"""
question = "What are transformers effective in?"
results = qa_pipeline(question=question,context=context)
results

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

Device set to use cpu


{'score': 0.6639722585678101,
 'start': 90,
 'end': 123,
 'answer': 'natural language processing tasks'}

In [24]:
results['answer']

'natural language processing tasks'

In [25]:
question=['What are transformers effective in?','What is transformers']
for q in question:
  results = qa_pipeline(question=q,context=context)
  print(f"{q} is \n {results['answer']}")

What are transformers effective in? is 
 natural language processing tasks
What is transformers is 
 a type of neural network architecture


In [26]:
import torch
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

In [27]:
model_name = "distilbert-base-uncased-distilled-squad"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

In [28]:
context = """
Transformers are a type of neural network architecture that has proven very effective
in natural language processing tasks such as translation, summarization, and question answering.
"""
question = "What are transformers effective in?"

In [29]:
inputs1 = tokenizer.encode_plus(question, context, return_tensors="pt")
inputs2 = tokenizer(question, context, return_tensors="pt")

In [30]:
print(inputs1)

{'input_ids': tensor([[  101,  2054,  2024, 19081,  4621,  1999,  1029,   102, 19081,  2024,
          1037,  2828,  1997, 15756,  2897,  4294,  2008,  2038, 10003,  2200,
          4621,  1999,  3019,  2653,  6364,  8518,  2107,  2004,  5449,  1010,
          7680,  7849,  3989,  1010,  1998,  3160, 10739,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [31]:
print(inputs2)

{'input_ids': tensor([[  101,  2054,  2024, 19081,  4621,  1999,  1029,   102, 19081,  2024,
          1037,  2828,  1997, 15756,  2897,  4294,  2008,  2038, 10003,  2200,
          4621,  1999,  3019,  2653,  6364,  8518,  2107,  2004,  5449,  1010,
          7680,  7849,  3989,  1010,  1998,  3160, 10739,  1012,   102]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}


In [32]:
with torch.no_grad():
    outputs = model(**inputs1)

In [33]:
print(outputs)

QuestionAnsweringModelOutput(loss=None, start_logits=tensor([[-3.6087, -1.3834, -4.4455, -5.1462, -5.5984, -5.7834, -6.7961, -4.1952,
          0.3752, -2.3047,  0.1792, -1.0289, -3.2155,  0.4019, -2.8337, -3.6773,
         -2.1924,  0.5996, -0.2567,  1.4552,  0.4001,  5.5784,  9.8733,  2.4162,
          1.8921,  1.9476, -0.3090, -0.5154,  5.0634, -2.5000,  0.5546, -3.5805,
         -3.7081, -4.5632, -1.7190,  3.8433, -0.6227, -2.0312, -4.1951]]), end_logits=tensor([[ 1.5828, -0.9164, -3.9716, -4.0415, -5.3970, -5.9616, -5.4072,  0.7678,
         -0.9782, -4.7669, -4.5158, -4.1299, -4.3215, -2.9225, -2.1382, -0.1710,
         -2.6407, -3.7716, -3.6661, -3.7771, -0.7429, -1.3387,  1.8818,  4.3482,
          5.9046,  9.8188,  1.5871, -0.2823,  3.6626,  0.5503, -1.6100, -1.6441,
          3.7606,  1.5100, -0.8276,  0.0966,  8.7454,  7.4695,  0.7681]]), hidden_states=None, attentions=None)


In [34]:
start_scores, end_scores = outputs.start_logits, outputs.end_logits

In [35]:
answer_start = torch.argmax(start_scores)
answer_end = torch.argmax(end_scores) + 1

In [43]:
inputs1["input_ids"]

tensor([[  101,  2054,  2024, 19081,  4621,  1999,  1029,   102, 19081,  2024,
          1037,  2828,  1997, 15756,  2897,  4294,  2008,  2038, 10003,  2200,
          4621,  1999,  3019,  2653,  6364,  8518,  2107,  2004,  5449,  1010,
          7680,  7849,  3989,  1010,  1998,  3160, 10739,  1012,   102]])

In [42]:
inputs1["input_ids"][0][1:9]

tensor([ 2054,  2024, 19081,  4621,  1999,  1029,   102, 19081])

In [44]:
answer = tokenizer.convert_tokens_to_string(
    tokenizer.convert_ids_to_tokens(inputs1["input_ids"][0][answer_start:answer_end])
)

In [45]:
answer

'natural language processing tasks'

In [1]:
!pip install datasets

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.8 MB/s eta 0:00:00


In [4]:
# ===========================
# 1️⃣ Imports
# ===========================
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForQuestionAnswering, TrainingArguments, Trainer
import torch
import evaluate #using evaulate as we need to check the persentage of accuracy per tokens like respones accurasy


In [5]:
# ===========================
# 2️⃣ Load dataset
# ===========================
dataset = load_dataset("squad")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

plain_text/train-00000-of-00001.parquet:   0%|          | 0.00/14.5M [00:00<?, ?B/s]

plain_text/validation-00000-of-00001.par(…):   0%|          | 0.00/1.82M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

In [6]:
# ===========================
# 3️⃣ Load tokenizer & model
# ===========================
model_name = "distilbert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForQuestionAnswering.from_pretrained(model_name)

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForQuestionAnswering were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [7]:
# ===========================
# 0 - Select subsets for training & validation
# ===========================
train_ds = dataset["train"].shuffle(seed=69).select(range(2000)) # we did this in Classification as well seed is random picks and range is well range
validation_ds = dataset['validation'].shuffle(seed=69).select(range(500))

In [8]:
import torch
device = "cuda" if torch.cuda.is_available() else "cpu" #as training on gpu we need to specify that train on gpu
print(f"Using device: {device}")
model.to(device)

Using device: cuda


DistilBertForQuestionAnswering(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): DistilBertSdpaAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
     

In [9]:
# ===========================
# 4️⃣ Preprocessing function (tokenize + labels)
# ===========================
def preprocess_function(examples):
    # Tokenize context + question with overlap
    tokenized_examples = tokenizer(
        examples["question"],
        examples["context"],
        truncation="only_second",
        max_length=384,
        stride=128,
        return_overflowing_tokens=True,
        return_offsets_mapping=True,
        padding="max_length"
    )
    #start of the token + label maker process from the dataset
    # Lists to store start/end positions
    start_positions = []
    end_positions = []

    # Map each chunk back to original example
    sample_mapping = tokenized_examples.pop("overflow_to_sample_mapping")

    for i, offsets in enumerate(tokenized_examples["offset_mapping"]):
        # Original example for this chunk
        sample_idx = sample_mapping[i]
        answer = examples["answers"][sample_idx]

        # If no answer, default to CLS token
        if len(answer["answer_start"]) == 0:
            start_positions.append(tokenizer.cls_token_id)
            end_positions.append(tokenizer.cls_token_id)
            continue

        answer_start_char = answer["answer_start"][0]
        answer_end_char = answer_start_char + len(answer["text"][0])

        # Find context token positions in this chunk
        sequence_ids = tokenized_examples.sequence_ids(i)
        cls_index = tokenized_examples["input_ids"][i].index(tokenizer.cls_token_id)

        token_start_index = 0
        while sequence_ids[token_start_index] != 1:
            token_start_index += 1
        token_end_index = len(tokenized_examples["input_ids"][i]) - 1
        while sequence_ids[token_end_index] != 1:
            token_end_index -= 1

        # If answer not fully inside chunk, label as CLS
        if not (offsets[token_start_index][0] <= answer_start_char and offsets[token_end_index][1] >= answer_end_char):
            start_positions.append(cls_index)
            end_positions.append(cls_index)
        else:
            # Exact token start
            while token_start_index < len(offsets) and offsets[token_start_index][0] <= answer_start_char:
                token_start_index += 1
            start_positions.append(token_start_index - 1)

            # Exact token end
            while token_end_index >= 0 and offsets[token_end_index][1] >= answer_end_char:
                token_end_index -= 1
            end_positions.append(token_end_index + 1)

    tokenized_examples["start_positions"] = start_positions
    tokenized_examples["end_positions"] = end_positions

    # We don’t need offset mapping for model
    tokenized_examples.pop("offset_mapping")
    return tokenized_examples

In [10]:
dataset["train"]

Dataset({
    features: ['id', 'title', 'context', 'question', 'answers'],
    num_rows: 87599
})

In [11]:
dataset["train"][2]

{'id': '5733be284776f41900661180',
 'title': 'University_of_Notre_Dame',
 'context': 'Architecturally, the school has a Catholic character. Atop the Main Building\'s gold dome is a golden statue of the Virgin Mary. Immediately in front of the Main Building and facing it, is a copper statue of Christ with arms upraised with the legend "Venite Ad Me Omnes". Next to the Main Building is the Basilica of the Sacred Heart. Immediately behind the basilica is the Grotto, a Marian place of prayer and reflection. It is a replica of the grotto at Lourdes, France where the Virgin Mary reputedly appeared to Saint Bernadette Soubirous in 1858. At the end of the main drive (and in a direct line that connects through 3 statues and the Gold Dome), is a simple, modern stone statue of Mary.',
 'question': 'The Basilica of the Sacred heart at Notre Dame is beside to which structure?',
 'answers': {'text': ['the Main Building'], 'answer_start': [279]}}

In [12]:
# ===========================
# 5️⃣ Apply preprocessing
# ===========================

#removing colomns to make it of same size we do this cuz of chunking
tokenized_train_ds = train_ds.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)
tokenized_validation_ds = validation_ds.map(
    preprocess_function,
    batched=True,
    remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/2000 [00:00<?, ? examples/s]

Map:   0%|          | 0/500 [00:00<?, ? examples/s]

In [13]:
tokenized_train_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 2025
})

In [14]:
print(tokenized_train_ds[2])

{'input_ids': [101, 5815, 2062, 3787, 2000, 1037, 8038, 5856, 1011, 20904, 2050, 2052, 2031, 2054, 3466, 1029, 102, 1037, 8038, 5856, 1011, 20904, 2050, 9140, 3594, 13135, 3787, 2000, 6551, 3623, 5114, 1012, 2009, 2003, 2328, 2247, 1037, 2490, 8797, 2008, 2003, 4197, 2646, 1996, 4742, 1010, 1998, 2947, 5927, 2053, 10572, 4742, 1998, 2515, 2025, 9002, 2000, 1996, 13438, 1005, 1055, 3169, 1012, 1996, 2203, 3553, 2000, 1996, 3120, 2003, 3615, 2000, 2004, 1996, 2392, 1012, 2379, 1996, 4373, 2003, 1037, 2309, 3161, 5783, 1010, 4050, 1037, 2431, 1011, 4400, 16510, 9890, 2030, 6999, 16510, 9890, 1012, 13135, 3787, 2024, 5412, 1999, 2392, 1006, 5501, 1007, 1998, 2369, 1006, 8339, 5668, 1007, 1996, 3161, 5783, 2247, 1996, 8797, 1012, 1996, 8038, 5856, 2038, 1996, 16112, 3737, 2008, 2009, 4150, 6233, 20396, 1010, 1998, 2947, 2038, 3020, 5114, 1010, 2004, 1996, 2193, 1997, 3787, 7457, 1012, 2174, 1010, 2023, 2036, 3084, 2009, 6233, 7591, 2000, 3431, 1999, 6075, 1025, 2065, 1996, 4742, 6075, 3431,

In [15]:
tokenized_validation_ds

Dataset({
    features: ['input_ids', 'attention_mask', 'start_positions', 'end_positions'],
    num_rows: 512
})

In [16]:
metric = evaluate.load("squad")
def compute_metrics(p):
    return metric.compute(predictions=p.predictions, references=p.label_ids)

In [17]:
training_args = TrainingArguments(
    output_dir="./qa_model",
    eval_strategy="steps",
    save_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    save_total_limit=2,
    logging_dir="./logs",
    push_to_hub=False,
    report_to="none",
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train_ds,
    eval_dataset=tokenized_validation_ds,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)


/tmp/ipython-input-2513484893.py:16: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [18]:
# Train
trainer.train()

# Save model
trainer.save_model("./custom_qa_model")
tokenizer.save_pretrained("./custom_qa_model")

Step,Training Loss,Validation Loss


('./custom_qa_model/tokenizer_config.json',
 './custom_qa_model/special_tokens_map.json',
 './custom_qa_model/vocab.txt',
 './custom_qa_model/added_tokens.json',
 './custom_qa_model/tokenizer.json')

In [19]:
hf_code = ''

In [20]:
from huggingface_hub import notebook_login
notebook_login()

In [21]:
name = './custom_qa_model'
model = AutoModelForQuestionAnswering.from_pretrained(name)
tokenizer = AutoTokenizer.from_pretrained(name)

# Push to Hub (your username/repo_name)
repo_name = "Noobhacker69/Custom_QA_model"  # you choose the name
model.push_to_hub(repo_name)
tokenizer.push_to_hub(repo_name)

model.safetensors:   0%|          | 0.00/265M [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

CommitInfo(commit_url='https://huggingface.co/Noobhacker69/Custom_QA_model/commit/97d9169d01dd10366f4dd4f149b92b8c748ca2e6', commit_message='Upload tokenizer', commit_description='', oid='97d9169d01dd10366f4dd4f149b92b8c748ca2e6', pr_url=None, repo_url=RepoUrl('https://huggingface.co/Noobhacker69/Custom_QA_model', endpoint='https://huggingface.co', repo_type='model', repo_id='Noobhacker69/Custom_QA_model'), pr_revision=None, pr_num=None)

In [1]:
from transformers import pipeline

In [4]:
repo_name = "Noobhacker69/Custom_QA_model"
qa_a = pipeline('question-answering',model=repo_name,tokenizer=repo_name)
context = """
Transformers are a type of neural network architecture that has proven very effective
in natural language processing tasks such as translation, summarization, and question answering.
"""
question = "What are transformers effective in?"
output = qa_a(question=question,context=context)
output

Fetching 0 files: 0it [00:00, ?it/s]

Fetching 1 files:   0%|          | 0/1 [00:00<?, ?it/s]

Fetching 0 files: 0it [00:00, ?it/s]

Device set to use cpu


{'score': 0.05956058204174042,
 'start': 28,
 'end': 55,
 'answer': 'neural network architecture'}